In [1]:
import os
import numpy as np
import imageio


In [2]:
from preprocessing.kitti_util import Calibration

In [3]:
def generate_disparity_from_velo(pc_velo, heigh, width, calib):
    pass

In [4]:
root_dir = 'C://users/cathx/repos/argoverse-api/argo_train/'

In [5]:
lidar_dir = root_dir + '/velodyne/'
calib_dir = root_dir + '/calib/'
image_dir = root_dir + '/image_2/'
disparity_dir = root_dir + '/disparity/'
split_file = root_dir + '/train.txt'

In [6]:
lidar_files = [lid for lid in os.listdir(lidar_dir) if lid[-3:]=='bin']
lidar_files = sorted(lidar_files)


In [7]:
print(len(lidar_files))

6988


In [8]:
with open(split_file, 'r') as f:
    file_names = [x.strip() for x in f.readlines()]
    
print(len(file_names))
# print(file_names)

6988


In [9]:
def generate_disparity_from_velo(pc_velo, heigh, width, calib):
    # first projects the velodyne images to reactangle
    # then uses the project_rect_to_image to change the nx3 points in rect camera coord
    # to nx2 points in the image coordinate.
    pts_2d = calib.project_velo_to_image(pc_velo)
    

In [10]:
for fn in lidar_files:
    # get rid of the .bin
    predix = fn[:-4]
    if predix not in file_names:
        continue
    calib_file = '{}/{}.txt'.format(calib_dir, predix)
    # loading the calibration files
    calib = Calibration(calib_file)
    
#     lidar = np.fromfile(lidar_dir + '/' + fn, dtype=np.float32).reshape((-1, 4))[:, :3]
# #     print(lidar)
# #     print(lidar.shape)
#     image_file = '{}/{}.png'.format(image_dir, predix)
#     image = imageio.imread(image_file)
#     height, width = image.shape[:2]
    

In [29]:
# read in the image
image_file = '{}/{}.png'.format(image_dir, predix)
image = imageio.imread(image_file)
height, width = image.shape[:2]
print(height, width)

2056 2464


In [26]:
predix = '000900'
calib_file = '{}/{}.txt'.format(calib_dir, predix)
calib = Calibration(calib_file)
print(calib)

In [21]:
lidar_o = np.fromfile(lidar_dir + '/' + '000100.bin', dtype=np.float32)
print(lidar_o)
print(len(lidar_o))

[ 0.7613791  -4.2393956  -0.43723643 ...  9.124742   -0.2929629
  0.        ]
346736


In [38]:
pts_2d = calib.project_velo_to_image(lidar)
print(pts_2d)
print(pts_2d.shape)

[[-18423.78910212  -7289.59735467]
 [-63932.66761158    365.49138163]
 [-22632.35084699    540.13722736]
 ...
 [  9139.28474256   1572.83920422]
 [ 11356.32074307  -1543.63263883]
 [ 10096.84369327   -447.26568085]]
(86684, 2)


In [41]:
# before we do project_velo_to_image or the project_velo_to_image uses the function 
# project_velo_to_rect
pts_rect = calib.project_velo_to_rect(lidar)
print(pts_rect)
print(pts_rect.shape)
print(pts_rect.dtype)


[[  4.36869542   1.84806998  -0.81616352]
 [  9.46079663   0.09683404  -0.53278692]
 [  9.30959261   0.19226467  -1.43227616]
 ...
 [-11.09884472  -0.75378506  -5.13784673]
 [ -8.08801083   2.05126919  -2.92565148]
 [ -8.95971748   1.49079423  -3.70050095]]
(86684, 3)
float64


In [42]:
# THEN, we do project_rect_to_image:
pts_image = calib.project_rect_to_image(pts_rect)
print(pts_image)
print(pts_image.shape)
print(pts_image.dtype)

[[-18423.78910212  -7289.59735467]
 [-63932.66761158    365.49138163]
 [-22632.35084699    540.13722736]
 ...
 [  9139.28474256   1572.83920422]
 [ 11356.32074307  -1543.63263883]
 [ 10096.84369327   -447.26568085]]
(86684, 2)
float64


In [40]:
lidar = np.fromfile(lidar_dir + '/' + '000100.bin', dtype=np.float32).reshape((-1, 4))[:, :3]
print(lidar_o)
print(len(lidar_o))
print(lidar.shape)
print(type(lidar))

[[ 0.7613791  -4.2393956  -0.43723643]
 [ 0.9657892  -9.310095    1.3856323 ]
 [ 0.06925678 -9.149933    1.2777976 ]
 ...
 [-3.4101672  11.310511    1.9058406 ]
 [-1.2012081   8.236688   -0.8326571 ]
 [-1.9722378   9.124742   -0.2929629 ]]
86684
(86684, 3)
<class 'numpy.ndarray'>


In [30]:
fov_inds = (pts_2d[:, 0] < width - 1) & (pts_2d[:, 0] >= 0) & \
               (pts_2d[:, 1] < height - 1) & (pts_2d[:, 1] >= 0)

In [32]:
fov_inds.shape

(86684,)

# Checking for issues with the Argo Data vs. KITTI

In [11]:
import imageio
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from preprocessing import kitti_util

In [2]:
argo_dir = 'C://users/cathx/repos/argoverse-api/argo_train'
kitti_dir = 'C://users/cathx/repos/PseudoLidar/pseudo_lidar/KITTI/object/training'

In [25]:
argo_calib = argo_dir + '/calib/'
argo_velodyne = argo_dir + '/velodyne/'
argo_image2 = argo_dir + '/image_2/'
argo_image3 = argo_dir + '/image_3/'
argo_label = argo_dir + '/label_2/'
argo_disp = argo_dir + '/disparity/'
argo_pdisp = argo_dir + '/predict_disparity'

In [73]:
kitti_calib = kitti_dir + '/calib/'
kitti_velodyne = kitti_dir + '/velodyne/'
kitti_image2 = kitti_dir + '/image_2/'
kitti_image3 = kitti_dir + '/image_3/'
kitti_label = kitti_dir + '/label_2/'
kitti_disp = kitti_dir + '/disparity/'

In [5]:
kimages = [x for x in os.listdir(kitti_image2) if x[-3:]=='png' or x[-3:] == 'npy']
aimages = [x for x in os.listdir(argo_image2) if x[-3:]=='png']


In [6]:
len(kimages)
len(aimages)

6988

### first:
- Load Images
- Load Calibration
- Load disparity files
- Load velodyne 

In [9]:
img = cv2.imread(os.path.join(kitti_image2 + '/' + '000000.png'))
# cv2.imshow('Image', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
img.shape

(370, 1224, 3)

In [10]:
img = cv2.imread(os.path.join(argo_image2 + '/' + '000000.png'))
# cv2.imshow('Image', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
img.shape

(2056, 2464, 3)

In [19]:
kitti_calib = open(os.path.join(kitti_calib + '/' + '000000.txt'))
# calib = kitti_util.Calibration(os.path.join(argo_calib + '/' + '000000.txt'))
kitti_calib.readlines()

['P0: 7.070493000000e+02 0.000000000000e+00 6.040814000000e+02 0.000000000000e+00 0.000000000000e+00 7.070493000000e+02 1.805066000000e+02 0.000000000000e+00 0.000000000000e+00 0.000000000000e+00 1.000000000000e+00 0.000000000000e+00\n',
 'P1: 7.070493000000e+02 0.000000000000e+00 6.040814000000e+02 -3.797842000000e+02 0.000000000000e+00 7.070493000000e+02 1.805066000000e+02 0.000000000000e+00 0.000000000000e+00 0.000000000000e+00 1.000000000000e+00 0.000000000000e+00\n',
 'P2: 7.070493000000e+02 0.000000000000e+00 6.040814000000e+02 4.575831000000e+01 0.000000000000e+00 7.070493000000e+02 1.805066000000e+02 -3.454157000000e-01 0.000000000000e+00 0.000000000000e+00 1.000000000000e+00 4.981016000000e-03\n',
 'P3: 7.070493000000e+02 0.000000000000e+00 6.040814000000e+02 -3.341081000000e+02 0.000000000000e+00 7.070493000000e+02 1.805066000000e+02 2.330660000000e+00 0.000000000000e+00 0.000000000000e+00 1.000000000000e+00 3.201153000000e-03\n',
 'R0_rect: 9.999128000000e-01 1.009263000000e

In [18]:
argo_calib = open(os.path.join(argo_calib + '/' + '000000.txt'))
# calib = kitti_util.Calibration(os.path.join(argo_calib + '/' + '000000.txt'))
argo_calib.readlines()

['P0: 0 0 0 0 0 0 0 0 0 0 0 0\n',
 'P1: 0 0 0 0 0 0 0 0 0 0 0 0\n',
 'P2: 3660.425411174431 0.0 1246.9202677240673 0.0 0.0 3660.425411174431 1063.4338217897928 0.0 0.0 0.0 1.0 0.0\n',
 'P3: 0 0 0 0 0 0 0 0 0 0 0 0\n',
 'R0_rect: 1 0 0 0 1 0 0 0 1\n',
 'Tr_velo_to_cam: 0.0011791067474549721 -0.9999732602806191 0.007217231534861368 0.13665588727286176 -0.00488923834063093 -0.007222915076280201 -0.9999619616996687 1.3737862567369516 0.9999873524478833 0.0011437751311049182 -0.0048976241916725005 -1.6132216033055031\n',
 'Tr_imu_to_velu: 0 0 0 0 0 0 0 0 0 0 0 0\n',
 '        ']

In [20]:
kitti_disp = np.load(os.path.join(kitti_disp + '/' + '000000.npy'))
kitti_disp

array([[-381.806622  , -381.806622  , -381.806622  , ..., -381.806622  ,
        -381.806622  , -381.806622  ],
       [-381.806622  , -381.806622  , -381.806622  , ..., -381.806622  ,
        -381.806622  , -381.806622  ],
       [-381.806622  , -381.806622  , -381.806622  , ..., -381.806622  ,
        -381.806622  , -381.806622  ],
       ...,
       [-381.806622  , -381.806622  , -381.806622  , ..., -381.806622  ,
        -381.806622  , -381.806622  ],
       [-381.806622  , -381.806622  , -381.806622  , ..., -381.806622  ,
          71.28876287, -381.806622  ],
       [-381.806622  , -381.806622  , -381.806622  , ..., -381.806622  ,
        -381.806622  , -381.806622  ]])

In [21]:
kitti_disp.shape

(370, 1224)

In [22]:
argo_disp = np.load(os.path.join(argo_disp + '/' + '000000.npy'))
argo_disp

array([[-1976.62972203, -1976.62972203, -1976.62972203, ...,
        -1976.62972203, -1976.62972203, -1976.62972203],
       [-1976.62972203, -1976.62972203, -1976.62972203, ...,
        -1976.62972203, -1976.62972203, -1976.62972203],
       [-1976.62972203, -1976.62972203, -1976.62972203, ...,
        -1976.62972203, -1976.62972203, -1976.62972203],
       ...,
       [-1976.62972203, -1976.62972203, -1976.62972203, ...,
        -1976.62972203, -1976.62972203, -1976.62972203],
       [-1976.62972203, -1976.62972203, -1976.62972203, ...,
        -1976.62972203, -1976.62972203, -1976.62972203],
       [-1976.62972203, -1976.62972203, -1976.62972203, ...,
        -1976.62972203, -1976.62972203, -1976.62972203]])

In [23]:
argo_disp.shape

(2056, 2464)

In [26]:
predicted_disp = np.load(os.path.join(argo_pdisp + '/' + '000000.npy'))
predicted_disp

array([[  6.0000463,   6.0000463,   6.000405 , ..., 181.       ,
        181.       , 181.       ],
       [  6.0000215,   6.0000215,   6.0001445, ..., 181.       ,
        181.       , 181.       ],
       [  6.000004 ,   6.000004 ,   6.0000186, ...,  10.01245  ,
         10.005265 ,  10.001251 ],
       ...,
       [ 21.960743 ,  21.960743 ,  21.98513  , ...,   5.9999704,
          5.9870358,   5.1450934],
       [ 22.002716 ,  22.002716 ,  22.014984 , ...,   5.9999948,
          5.998748 ,   5.76061  ],
       [ 22.002716 ,  22.002716 ,  22.014984 , ...,   5.9999948,
          5.998748 ,   5.76061  ]], dtype=float32)

In [27]:
predicted_disp.shape

(383, 1245)

In [28]:
# no bueno. No Bueno

# Generating Lidar

In [29]:
argo_dir = 'C://users/cathx/repos/argoverse-api/argo_train'

argo_calib = argo_dir + '/calib/'
argo_velodyne = argo_dir + '/velodyne/'
argo_image2 = argo_dir + '/image_2/'
argo_image3 = argo_dir + '/image_3/'
argo_label = argo_dir + '/label_2/'
argo_disp = argo_dir + '/disparity/'
argo_pdisp = argo_dir + '/p_disparity'
split_file = argo_dir + '/train.txt'

In [31]:
if not os.path.isdir(argo_pdisp):
    os.makedirs(argo_pdisp)

In [32]:
# load the lidar files:
lidar = [x for x in os.listdir(argo_velodyne) if x [-3:] == 'bin']
lidar = sorted(lidar)

In [33]:
assert os.path.isfile(split_file)
with open(split_file, 'r') as f:
    filenames = [x.strip() for x in f.readlines()]
# filenames[:10]

In [ ]:
for fn in lidar:
    predix = fn[:-4]
    if predix not in filenames:
        continue
    
    calib_file = '{}/{}.txt'.format(argo_calib, predix)
    calib = kitti_util.Calibration(calib_file)
    lidar = np.fromfile(argo_velodyne + '/' + fn, dtype=np.float32).reshape((-1, 4))[:, :3]
    image_file = '{}/{}.png'.format(image_dir, predix)
    image = imageio.imread(image_file)
    height, width = image.shape[:2]
    disp = generate_disparity_from_velo(lidar, height, width, calib)
    np.save(disparity_dir + '/' + predix, disp)
    print('Finished Disparity {}'.format(predix))

In [ ]:
def generate_disparity_from_velo(lidar, height, width, calib):
    pts_2d = calib.project_velo_to_image(lidar)
    fov_inds = (pts_2d[:, 0] < width -1) & (pts_2d[:,0] >= 0) & \
                (pts_2d[:, 1] < height - 1) * (pts_2d[:,1] >= 0)
    fov_inds

In [35]:
calib_file = '{}/{}.txt'.format(argo_calib, '000000')
calib = kitti_util.Calibration(calib_file)

In [74]:
calib_file = '{}/{}.txt'.format(kitti_calib, '000000')
calib = kitti_util.Calibration(calib_file)

In [81]:
lidar = np.fromfile(argo_velodyne + '/' + '000000.bin', dtype=np.float32).reshape((-1, 4))[:, :3]

In [82]:
k_lidar = np.fromfile(kitti_velodyne + '/' + '000000.bin', dtype=np.float32).reshape((-1, 4))[:, :3]

In [83]:
lidar.shape

(86185, 3)

In [84]:
k_lidar.shape

(115384, 3)

In [76]:
pts_2d = calib.project_velo_to_image(lidar)
print(pts_2d)

[[602.0853193  141.7459889 ]
 [599.8489135  141.81345282]
 [596.12144246 149.02292823]
 ...
 [907.61576863 520.91062242]
 [904.84647192 520.52796994]
 [900.24350935 520.43991183]]


In [41]:
pts_2d.shape

(86185, 2)

In [77]:
pts_2d.shape

(115384, 2)

In [43]:
lidar.shape

(86185, 3)

In [78]:
lidar.shape

(115384, 3)

In [45]:
n = lidar.shape[0]
n

86185

In [89]:
pts_3d = np.hstack((lidar, np.ones((n, 1))))
# pts_3d

In [80]:
pts_3d.shape

(86185, 4)

In [59]:
V2C = [[0.0011791067474549721, -0.9999732602806191, 0.007217231534861368, 0.13665588727286176],
       [-0.00488923834063093, -0.007222915076280201, -0.9999619616996687, 1.3737862567369516],
       [0.9999873524478833, 0.0011437751311049182, -0.0048976241916725005, -1.6132216033055031]]

In [56]:
calib_file = '{}/{}.txt'.format(argo_calib, '000000')
calibs = open(calib_file)
calibs = calibs.readlines()
calibs
# V2C = calibs['TR_velo_to_cam']
# V2C?

['P0: 0 0 0 0 0 0 0 0 0 0 0 0\n',
 'P1: 0 0 0 0 0 0 0 0 0 0 0 0\n',
 'P2: 3660.425411174431 0.0 1246.9202677240673 0.0 0.0 3660.425411174431 1063.4338217897928 0.0 0.0 0.0 1.0 0.0\n',
 'P3: 0 0 0 0 0 0 0 0 0 0 0 0\n',
 'R0_rect: 1 0 0 0 1 0 0 0 1\n',
 'Tr_velo_to_cam: 0.0011791067474549721 -0.9999732602806191 0.007217231534861368 0.13665588727286176 -0.00488923834063093 -0.007222915076280201 -0.9999619616996687 1.3737862567369516 0.9999873524478833 0.0011437751311049182 -0.0048976241916725005 -1.6132216033055031\n',
 'Tr_imu_to_velu: 0 0 0 0 0 0 0 0 0 0 0 0\n',
 '        ']

In [60]:
output_velo_to_ref = np.dot(pts_3d, np.transpose(V2C))
output_velo_to_ref

array([[ 3.32243457,  1.32410844, -0.62892848],
       [ 9.3925415 ,  0.04693034, -0.43283028],
       [ 9.58285897,  0.1703532 , -1.36353465],
       ...,
       [-8.02896536,  2.14747973, -2.97541574],
       [-8.55145015,  1.54158231, -3.63269774],
       [-8.56561121, -0.37380738, -3.17267264]])

In [61]:
output_velo_to_ref.shape

(86185, 3)

In [62]:
R0 = [[1, 0, 0],
[0, 1, 0],
[0, 0, 1]]

In [63]:
# project_ref_to_rect
final_rect = np.transpose(np.dot(R0, np.transpose(output_velo_to_ref)))

In [64]:
final_rect

array([[ 3.32243457,  1.32410844, -0.62892848],
       [ 9.3925415 ,  0.04693034, -0.43283028],
       [ 9.58285897,  0.1703532 , -1.36353465],
       ...,
       [-8.02896536,  2.14747973, -2.97541574],
       [-8.55145015,  1.54158231, -3.63269774],
       [-8.56561121, -0.37380738, -3.17267264]])

In [65]:
final_rect.shape

(86185, 3)

In [85]:
new_disp = np.load(os.path.join(argo_dir + '/pred_disparity/000000.npy'))
new_disp.shape

(383, 1245)

In [88]:
new_disp

array([[  6.0000463,   6.0000463,   6.000405 , ..., 181.       ,
        181.       , 181.       ],
       [  6.0000215,   6.0000215,   6.0001445, ..., 181.       ,
        181.       , 181.       ],
       [  6.000004 ,   6.000004 ,   6.0000186, ...,  10.01245  ,
         10.005265 ,  10.001251 ],
       ...,
       [ 21.960743 ,  21.960743 ,  21.98513  , ...,   5.9999704,
          5.9870358,   5.1450934],
       [ 22.002716 ,  22.002716 ,  22.014984 , ...,   5.9999948,
          5.998748 ,   5.76061  ],
       [ 22.002716 ,  22.002716 ,  22.014984 , ...,   5.9999948,
          5.998748 ,   5.76061  ]], dtype=float32)

In [86]:
old_disp = np.load(os.path.join(kitti_dir + '/disparity/000000.npy'))
old_disp

array([[-381.806622  , -381.806622  , -381.806622  , ..., -381.806622  ,
        -381.806622  , -381.806622  ],
       [-381.806622  , -381.806622  , -381.806622  , ..., -381.806622  ,
        -381.806622  , -381.806622  ],
       [-381.806622  , -381.806622  , -381.806622  , ..., -381.806622  ,
        -381.806622  , -381.806622  ],
       ...,
       [-381.806622  , -381.806622  , -381.806622  , ..., -381.806622  ,
        -381.806622  , -381.806622  ],
       [-381.806622  , -381.806622  , -381.806622  , ..., -381.806622  ,
          71.28876287, -381.806622  ],
       [-381.806622  , -381.806622  , -381.806622  , ..., -381.806622  ,
        -381.806622  , -381.806622  ]])

In [87]:
old_disp.shape

(370, 1224)

In [90]:
pseudo_dir = 'C://users/cathx/repos/PseudoLidar/pseudo_lidar/'
lidar = np.load(pseudo_dir + '/pred_disparity/000000.npy')
lidar

array([[ 6.000043 ,  6.000043 ,  6.0001287, ..., 25.094204 , 25.361883 ,
        25.759272 ],
       [ 6.0000286,  6.0000286,  6.000063 , ..., 25.046757 , 25.242401 ,
        25.67875  ],
       [ 6.0006027,  6.0006027,  6.0000534, ..., 24.99519  , 25.08181  ,
        25.482426 ],
       ...,
       [72.44437  , 72.44437  , 72.47504  , ...,  9.355137 ,  9.181206 ,
         9.055973 ],
       [72.79576  , 72.79576  , 72.819565 , ..., 29.963627 , 10.169769 ,
         9.196623 ],
       [72.79576  , 72.79576  , 72.819565 , ..., 29.963627 , 10.169769 ,
         9.196623 ]], dtype=float32)

In [95]:
lidar.shape

(383, 1245)

In [91]:
lidar2 = np.load(argo_dir + '/predict_disparity/000000.npy')
lidar2

array([[  6.0000463,   6.0000463,   6.000405 , ..., 181.       ,
        181.       , 181.       ],
       [  6.0000215,   6.0000215,   6.0001445, ..., 181.       ,
        181.       , 181.       ],
       [  6.000004 ,   6.000004 ,   6.0000186, ...,  10.01245  ,
         10.005265 ,  10.001251 ],
       ...,
       [ 21.960743 ,  21.960743 ,  21.98513  , ...,   5.9999704,
          5.9870358,   5.1450934],
       [ 22.002716 ,  22.002716 ,  22.014984 , ...,   5.9999948,
          5.998748 ,   5.76061  ],
       [ 22.002716 ,  22.002716 ,  22.014984 , ...,   5.9999948,
          5.998748 ,   5.76061  ]], dtype=float32)

In [92]:
lidar2.shape

(383, 1245)

In [11]:
from preprocessing import kitti_util
import numpy as np

In [4]:
argo_calib = 'C://users/cathx/repos/argoverse-api/argo_train/calib/'
calib_file = '{}/{}.txt'.format(argo_calib, '000000')
calib = kitti_util.Calibration(calib_file)

In [13]:
argo_dir = 'C://users/cathx/repos/argoverse-api/argo_train/'

In [42]:
lidar_dir = argo_dir + '/velodyne'
lidar_file
lidar = np.fromfile(lidar_dir + '/' + '000000.bin', dtype=np.float32).reshape((-1, 4))[:, :3]

In [43]:
lidar

array([[ 0.98827994, -3.1842089 ,  0.06784773],
       [ 1.1977774 , -9.244704  ,  1.3878262 ],
       [ 0.26670575, -9.436973  ,  1.2703398 ],
       ...,
       [-1.3755878 ,  8.158257  , -0.8259257 ],
       [-2.0305152 ,  8.684352  , -0.22060311],
       [-1.5611478 ,  8.712873  ,  1.6923585 ]], dtype=float32)

In [20]:
lidar.shape

(86185, 3)

In [18]:
pts_2d = calib.project_velo_to_image(lidar)
pts_2d

array([[-18089.97480067,  -6643.00718092],
       [-78185.3642061 ,    666.54615094],
       [-24478.38167794,    606.11858806],
       ...,
       [ 11124.3395477 ,  -1578.44551779],
       [  9863.64196997,   -489.91508478],
       [ 11129.37097472,   1494.70871345]])

In [21]:
pts_2d.shape

(86185, 2)

In [ ]:
# for project_velo_to_image
# project_velo_to_rect -> project_velo_to_ref 
# return project_rect_to_image


In [40]:
lidar.shape

(86185, 3)

In [44]:
lidar

array([[ 0.98827994, -3.1842089 ,  0.06784773],
       [ 1.1977774 , -9.244704  ,  1.3878262 ],
       [ 0.26670575, -9.436973  ,  1.2703398 ],
       ...,
       [-1.3755878 ,  8.158257  , -0.8259257 ],
       [-2.0305152 ,  8.684352  , -0.22060311],
       [-1.5611478 ,  8.712873  ,  1.6923585 ]], dtype=float32)

In [36]:
# V2C is calibs['Tr_velo_to_cam']
V2C = [0.0011791067474549721, -0.9999732602806191, 0.007217231534861368, 0.13665588727286176, -0.00488923834063093, -0.007222915076280201, -0.9999619616996687, 1.3737862567369516, 0.9999873524478833, 0.0011437751311049182, -0.0048976241916725005, -1.6132216033055031]
V2C = np.reshape(V2C, [3,4])
V2C

array([[ 1.17910675e-03, -9.99973260e-01,  7.21723153e-03,
         1.36655887e-01],
       [-4.88923834e-03, -7.22291508e-03, -9.99961962e-01,
         1.37378626e+00],
       [ 9.99987352e-01,  1.14377513e-03, -4.89762419e-03,
        -1.61322160e+00]])

In [37]:
cart2hom = calib.cart2hom(lidar)
cart2hom.shape

(86185, 4)

In [38]:
transposed_cart2hom = np.dot(cart2hom, np.transpose(V2C))
transposed_cart2hom

array([[ 3.32243457,  1.32410844, -0.62892848],
       [ 9.3925415 ,  0.04693034, -0.43283028],
       [ 9.58285897,  0.1703532 , -1.36353465],
       ...,
       [-8.02896536,  2.14747973, -2.97541574],
       [-8.55145015,  1.54158231, -3.63269774],
       [-8.56561121, -0.37380738, -3.17267264]])

In [39]:
transposed_cart2hom.shape


(86185, 3)

In [55]:
tr = np.transpose(transposed_cart2hom)
tr.shape

(3, 86185)

R0 = [1, 0, 0, 0, 1, 0, 0, 0, 1]
R0 = np.reshape(R0, [3,3])

In [51]:
ref_to_rect = np.transpose(np.dot(R0, np.transpose(transposed_cart2hom)))


In [52]:
ref_to_rect

array([[ 3.32243457,  1.32410844, -0.62892848],
       [ 9.3925415 ,  0.04693034, -0.43283028],
       [ 9.58285897,  0.1703532 , -1.36353465],
       ...,
       [-8.02896536,  2.14747973, -2.97541574],
       [-8.55145015,  1.54158231, -3.63269774],
       [-8.56561121, -0.37380738, -3.17267264]])

In [53]:
ref_to_rect.shape

(86185, 3)

In [1]:
s = 'Elon Musk'
l = len(s)
print(l)

9
